In [ ]:
import collections
import heapq

# --- Task 7: Huffman Coding Implementation ---

class HuffmanNode:
    """Represents a node in the Huffman tree."""
    def __init__(self, char, freq):
        self.char = char  # Character (None for internal nodes)
        self.freq = freq  # Frequency
        self.left = None  # Left child node
        self.right = None # Right child node

    # For use with heapq (priority queue)
    def __lt__(self, other):
        return self.freq < other.freq

    def __repr__(self):
        return f"Node({self.char if self.char else 'Internal'}, {self.freq})"

def build_huffman_tree(text):
    """
    Builds the Huffman tree and returns its root.
    """
    # 1. Count character frequencies
    frequencies = collections.Counter(text)

    # 2. Create a list of HuffmanNodes for leaf nodes and put them in a min-heap (priority queue)
    priority_queue = [HuffmanNode(char, freq) for char, freq in frequencies.items()]
    heapq.heapify(priority_queue)

    # 3. Build the Huffman tree
    while len(priority_queue) > 1:
        # Extract the two nodes with the lowest frequencies
        left_child = heapq.heappop(priority_queue)
        right_child = heapq.heappop(priority_queue)

        # Create a new internal node
        parent_freq = left_child.freq + right_child.freq
        parent_node = HuffmanNode(None, parent_freq)
        parent_node.left = left_child
        parent_node.right = right_child

        # Add the new internal node back to the priority queue
        heapq.heappush(priority_queue, parent_node)

    return priority_queue[0] if priority_queue else None # The root of the Huffman tree

def generate_huffman_codes(node, current_code="", codes={}):
    """
    Generates Huffman codes by traversing the Huffman tree.
    """
    if node is None:
        return codes

    # If it's a leaf node, we've found a character's code
    if node.char is not None:
        codes[node.char] = current_code
        return codes

    # Traverse left (add '0') and right (add '1')
    generate_huffman_codes(node.left, current_code + "0", codes)
    generate_huffman_codes(node.right, current_code + "1", codes)
    return codes

def encode_text(text, huffman_codes):
    """
    Encodes the given text using the generated Huffman codes.
    """
    encoded_bits = ""
    for char in text:
        encoded_bits += huffman_codes[char]
    return encoded_bits

def decode_text(encoded_bits, huffman_tree_root):
    """
    Decodes a bit string using the Huffman tree.
    """
    decoded_text = []
    current_node = huffman_tree_root

    for bit in encoded_bits:
        if bit == '0':
            current_node = current_node.left
        else: # bit == '1'
            current_node = current_node.right

        if current_node.char is not None: # Reached a leaf node (character)
            decoded_text.append(current_node.char)
            current_node = huffman_tree_root # Reset to root for the next character

    return "".join(decoded_text)

def print_huffman_tree_structure(node, indent=0):
    """Helper function to print the Huffman tree structure."""
    if node is None:
        return
    prefix = "  " * indent
    print(f"{prefix}{node.char if node.char else 'Internal Node'} (Freq: {node.freq})")
    if node.left or node.right:
        print(f"{prefix}  L:", end="")
        print_huffman_tree_structure(node.left, indent + 1)
        print(f"{prefix}  R:", end="")
        print_huffman_tree_structure(node.right, indent + 1)


print("--- ВИКОНАННЯ ЗАВДАННЯ З КОДУВАННЯМ ХАФФМАНА (З ПЕРШОГО ФОТО) ---")

text_to_encode = "AABAABACBBCBCEEFFFGGGLLLRRRTTGGGDDEE"
print(f"Початковий текст: '{text_to_encode}'")
print(f"Довжина початкового тексту (символів): {len(text_to_encode)}")

# 1. Build Huffman Tree
huffman_tree_root = build_huffman_tree(text_to_encode)

# 2. Generate Huffman Codes
huffman_codes = generate_huffman_codes(huffman_tree_root)

print("\nЧастоти символів та згенеровані коди Хаффмана:")
total_bits_huffman = 0
unique_chars = sorted(list(collections.Counter(text_to_encode).keys())) # Sort for consistent output
for char in unique_chars:
    freq = collections.Counter(text_to_encode)[char]
    code = huffman_codes[char]
    total_bits_huffman += freq * len(code)
    print(f"  Символ: '{char}', Частота: {freq}, Код: '{code}' (Довжина: {len(code)})")

print(f"\nЗагальна кількість бітів після кодування Хаффмана: {total_bits_huffman} бітів")

# 3. Assess compression effect vs. non-optimal (fixed-length) case
num_unique_chars = len(unique_chars)
bits_per_char_fixed = (num_unique_chars - 1).bit_length() # ceil(log2(num_unique_chars))

if num_unique_chars <= 1: # Handle cases with 0 or 1 unique char
    bits_per_char_fixed = 1 if num_unique_chars == 1 else 0

total_bits_fixed_length = len(text_to_encode) * bits_per_char_fixed

print("\nОцінка ефекту від кодування:")
print(f"Кількість унікальних символів: {num_unique_chars}")
print(f"Кількість бітів на символ для неоптимального (фіксованого) кодування: {bits_per_char_fixed} бітів")
print(f"Загальна кількість бітів для неоптимального кодування: {total_bits_fixed_length} бітів")

print(f"\nЕкономія бітів за допомогою кодування Хаффмана: {total_bits_fixed_length - total_bits_huffman} бітів")
if total_bits_fixed_length > 0:
    compression_ratio = (total_bits_fixed_length - total_bits_huffman) / total_bits_fixed_length * 100
    print(f"Відсоток стиснення: {compression_ratio:.2f}%")
else:
    print("Немає стиснення, оскільки початковий текст порожній.")


print("\nСтруктура двійкового дерева Хаффмана (корінь зверху, ліво = 0, право = 1):")
print_huffman_tree_structure(huffman_tree_root)

# Verify decoding
encoded_text = encode_text(text_to_encode, huffman_codes)
decoded_text = decode_text(encoded_bits, huffman_tree_root)
print(f"\nЗакодований текст (перші 100 бітів): {encoded_text[:100]}...")
print(f"Декодований текст: '{decoded_text}'")
print(f"Декодування успішне: {decoded_text == text_to_encode}")


print("\n" + "="*70 + "\n")

# --- КОНТРОЛЬНІ ЗАПИТАННЯ (З ДРУГОГО ФОТО) ---

print("--- ВІДПОВІДІ НА КОНТРОЛЬНІ ЗАПИТАННЯ ---")

# Question 1
print("\n1. Що таке кодування Гафмена та як воно працює?")
print("""
Кодування Хаффмана (Huffman coding) - це жадібний алгоритм безвтратного стиснення даних, який розробляє оптимальний префіксний код. Він працює на основі частоти появи символів у вхідному потоці даних: частіше зустрічаються символам присвоюються коротші двійкові коди, а рідше зустрічаються - довші. Це дозволяє досягти високого ступеня стиснення для даних з нерівномірним розподілом частот символів.

Принцип роботи:
1. Підрахунок частот: Для кожного унікального символу визначається його частота.
2. Пріоритетна черга: Символи поміщаються в пріоритетну чергу як "листові" вузли (пріоритет за частотою).
3. Побудова дерева Хаффмана: На кожному кроці два вузли з найменшими частотами об'єднуються в новий внутрішній вузол, і новий вузол додається до черги. Процес триває до одного кореневого вузла.
4. Присвоєння кодів: Двійкові коди генеруються обходом дерева від кореня до листових вузлів (ліво = '0', право = '1').
""")

# Question 2
print("\n2. Як визначається оптимальний двійковий код Гафмена для стиснення даних?")
print("""
Оптимальний двійковий код Хаффмана визначається шляхом побудови дерева Хаффмана. Ключовим принципом оптимальності є те, що частіше зустрічаються символи отримують коротші коди, а рідше - довші.

Оптимальність гарантується жадібним підходом: на кожному кроці об'єднуються два найменш частотні символи (або піддерева). Це забезпечує, що ці символи будуть на найбільшій глибині дерева (найдовші коди), тоді як символи з більшою частотою будуть розташовані ближче до кореня (коротші коди), що мінімізує загальну довжину закодованого повідомлення.

Код Хаффмана є префіксним кодом, що означає, що жоден код символу не є префіксом іншого, забезпечуючи однозначне декодування.
""")

# Question 3
print("\n3. Які переваги має кодування Гафмена над іншими методами стиснення даних?")
print("""
Переваги кодування Хаффмана:
* Оптимальність для префіксних кодів: Є оптимальним серед усіх префіксних кодів.
* Безвтратне стиснення: Дані повністю відновлюються.
* Гнучкість: Адаптується до розподілу частот у конкретних даних.
* Простота реалізації та швидкість: Відносно простий і швидкий.
* Універсальність: Може бути застосований до будь-яких дискретних символів.
""")

# Question 4
print("\n4. Як відбувається декодування даних, закодованих за допомогою кодування Гафмена?")
print("""
Декодування даних, закодованих Хаффманом, відбувається за допомогою побудованого дерева Хаффмана:
1. Наявність дерева: Необхідно мати те саме дерево (або мапу кодів), що було використано для кодування.
2. Покроковий обхід: Зчитується бітовий потік по одному біту, обходячи дерево від кореня: '0' -> ліво, '1' -> право.
3. Ідентифікація символу: При досягненні листового вузла (символу), символ додається до декодованого виводу.
4. Скидання до кореня: Після ідентифікації символу обхід починається знову з кореня.
5. Завершення: Декодування триває до кінця бітового потоку.
""")

# Question 5
print("\n5. Які є можливі недоліки кодування Гафмена?")
print("""
Можливі недоліки кодування Хаффмана:
* Потрібна таблиця кодів/дерево: Додає накладні витрати для коротких повідомлень.
* Двоетапний процес: Потрібно два проходи по даних (підрахунок частот, потім кодування).
* Неефективність для рівномірного розподілу: Не дає значного стиснення, якщо частоти символів приблизно однакові.
* Символьна основа: Не враховує контекст або повторювані послідовності (фрази).
* Чутливість до змін даних: Статичне кодування може бути менш ефективним для файлів зі змінними частотами символів.
""")

# Question 6
print("\n6. Для чого використовується побудова дерева в кодуванні Гафмена?")
print("""
Побудова дерева в кодуванні Хаффмана використовується для:
* Визначення оптимальних префіксних кодів: Шляхи від кореня до листів визначають унікальні двійкові коди.
* Забезпечення властивості префіксу: Жоден код не є префіксом іншого, що гарантує однозначне декодування.
* Зручність декодування: Дерево є ефективною структурою для обходу бітового потоку та ідентифікації символів.
* Відображення ієрархії частот: Візуалізує, як символи з різними частотами розташовані в структурі коду.
""")